**This notebook contains codes to generate inputs for CellPhoneDB and prepare the results table from CellPhoneDB**

In [ ]:
import sys
sys.path.append("../code")
from data_prep.cellphone import export_for_cellphonedb, load_cellphonedb_results, annotate_cellphone_results

In [1]:
import scanpy as sc
from pathlib import Path
import numpy as np
import pandas as pd
import scrublet as scr

import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sc.settings.verbosity=3

In [2]:
main_dir = "/projects/robson-lab/research/endometriosis/"
sample_id = "Endometriosis-revision-3"

sc.settings.figdir = f"{main_dir}figures/{sample_id}"
sc.set_figure_params(dpi=150, color_map='Reds')
sns.set_style("white")

In [3]:
import sys
sys.path.append("data_prep")

from cellphone import export_for_cellphonedb

In [4]:
#main_cluster = sc.read(f"{main_dir}data/h5ad/{sample_id}/{sample_id}-scrubs-clean-final-20211118.h5ad")

In [4]:
#load adatas
mye = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Myeloid/{sample_id}-mye-final-cc-20220105.h5ad")
lym = sc.read(f"{main_dir}/data/h5ad/{sample_id}/TNK/{sample_id}-lym-final-cc-20220105.h5ad")
epi = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Epithelial/{sample_id}-epi-final-cc-20220119.h5ad")
stro = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Stromal/{sample_id}-stro-final-cc-20220105.h5ad")
endo = sc.read(f"{main_dir}/data/h5ad/{sample_id}/Endothelial/{sample_id}-endo-final-cc-20220105.h5ad")

#combined all data
adatas = mye.concatenate([lym, epi, stro, endo])

In [5]:
#create cellphoneDB inputs
export_for_cellphonedb(adatas,outdir = f"{main_dir}analysis/{sample_id}/cellphone/all-celltypes-2",
                      celltype_key="subtypes",
                      partition_key="sampleid", partition_prefix="",
                      use_raw = False, layer= "raw"
                      )

---
_run cellphoneDB on terminal, in the dir where the output will be saved_

```
for d in $(find * -type f -name "*_counts.txt"); do 
    sbatch /projects/robson-lab/research/endometriosis/notebooks/EC19001-EC20015/scripts/run_cellphonedb.sbatch ${d%_*}; 
    done
```
_once finished, load in the results_

this takes about 9 hours

---

In [7]:
results = load_cellphonedb_results(adatas, results_dir = f"{main_dir}analysis/{sample_id}/cellphone/all-celltypes-2",
                                   partition_key="sampleid", partition_prefix="", use_raw=False, layer="raw")
ann_results = annotate_cellphone_results(results)

In [ ]:
results.to_csv(f"{main_dir}/analysis/{sample_id}/cellphone/all-celltypes-2/annotated_results_20220125.csv")

--- End of process --